# Creating a database of scalar values for ICRH coupling analysis

In [1]:
# assume working in Jupyter Lab
%matplotlib inline 

%load_ext autoreload
%autoreload 2

In [2]:
import pywed as pw
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = (10,6)

from control_room import *
from pulse_database import PulseDB
from tqdm import tqdm

import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')

The database has been created in another notebook. Importing database : 

In [3]:
hdf5_filename = 'databases/WEST_C3b_ICRH_pulse_data.hdf5'
db = PulseDB(hdf5_filename)
print(f'Database contains {len(db.pulse_list)} shots, from #{db.pulse_list[0]} to #{db.pulse_list[-1]} ')

Database contains 131 shots, from #53566 to #54178 


## Creating a meaningfull database with pandas
The idea is to split time in small pieces and to calculate scalar values for each of them

In [4]:
def split_in_pieces(y, t, nb_pieces):
    """Split a time signel y(t) into smaller piece of length dt, and return t, average, min, max and std of each of them"""
    y_mean_min_max, t_pieces = [], []
    if nb_pieces > 0: 
        ts = np.array_split(np.squeeze(t), nb_pieces)
        ys = np.array_split(np.squeeze(y), nb_pieces)
        for (_y, _t) in zip(ys, ts):
            t_pieces.append(np.mean(_t))
            y_mean_min_max.append(mean_min_max(_y))
        return np.array(y_mean_min_max), np.array(t_pieces)    
    else:
        return np.array([np.nan, np.nan, np.nan]), np.array([np.nan])
    

In [6]:
DELTA_T_BEFORE = 1  # s
dt = 0.05  # s

data_Q1 = pd.DataFrame()

# generate averaged values for Q1
for pulse in tqdm(db.pulse_list):
    try:
        P_IC, t_P_IC = db.get_signal(pulse, 'IC_P_Q1')
        Rc,   t_Rc   = db.get_signal(pulse, 'IC_Rc_Q1_avg')
        V7,  t_V7  = db.get_signal(pulse, 'Valve7')
        V9,  t_V9  = db.get_signal(pulse, 'Valve9')
        V10,  t_V10  = db.get_signal(pulse, 'Valve10')
        V11,  t_V11  = db.get_signal(pulse, 'Valve11')
        V21,  t_V21  = db.get_signal(pulse, 'Valve21')
        nl, t_nl = db.get_signal(pulse, 'nl')
        ip, t_ip = db.get_signal(pulse, 'Ip')
        
        try:  # il n'y a pas forcément de P_LH
            P_LH, t_P_LH = db.get_signal(pulse, 'LH_P_tot')
            if np.any(np.isnan(P_LH)):
                P_LH = np.zeros_like(P_IC)
                t_P_LH = t_P_IC
        except:
            P_LH = np.zeros_like(P_IC)
            t_P_LH = t_P_IC
        
        Rext_upper, t_Rext_upper = db.get_signal(pulse, 'Rext_upper')
        Rext_median, t_Rext_median = db.get_signal(pulse, 'Rext_median')
        Rext_lower, t_Rext_lower = db.get_signal(pulse, 'Rext_lower')
        Zgeo, t_Zgeo = db.get_signal(pulse, 'Zgeo')
        IC_Positions, _ = db.get_signal(pulse, 'IC_Positions')
        IC_Frequencies, _ = np.round(db.get_signal(pulse, 'IC_Frequencies'), decimals=1)
        
        C_LU, t_C_LU = db.get_signal(pulse, 'IC_Capa_Q1_left_upper')
        C_LL, t_C_LL = db.get_signal(pulse, 'IC_Capa_Q1_left_lower')
        C_RU, t_C_RU = db.get_signal(pulse, 'IC_Capa_Q1_right_upper')
        C_RL, t_C_RL = db.get_signal(pulse, 'IC_Capa_Q1_right_lower')

        try:
            Lang1, t_Lang1 = db.get_signal(pulse, 'Langmuir_LHCD1')
            Lang2, t_Lang2 = db.get_signal(pulse, 'Langmuir_LHCD2')
            Lang3, t_Lang3 = db.get_signal(pulse, 'Langmuir_LHCD3')
            Lang4, t_Lang4 = db.get_signal(pulse, 'Langmuir_LHCD4')
            Lang5, t_Lang5 = db.get_signal(pulse, 'Langmuir_LHCD5')
            Lang6, t_Lang6 = db.get_signal(pulse, 'Langmuir_LHCD6')
            Lang7, t_Lang7 = db.get_signal(pulse, 'Langmuir_LHCD7')
            Lang8, t_Lang8 = db.get_signal(pulse, 'Langmuir_LHCD8')
        except:
            Lang1, t_Lang1 = np.zeros_like(P_IC), t_P_IC
            Lang2, t_Lang2 = np.zeros_like(P_IC), t_P_IC
            Lang3, t_Lang3 = np.zeros_like(P_IC), t_P_IC
            Lang4, t_Lang4 = np.zeros_like(P_IC), t_P_IC
            Lang5, t_Lang5 = np.zeros_like(P_IC), t_P_IC
            Lang6, t_Lang6 = np.zeros_like(P_IC), t_P_IC
            Lang7, t_Lang7 = np.zeros_like(P_IC), t_P_IC
            Lang8, t_Lang8 = np.zeros_like(P_IC), t_P_IC
        
        t_start = db.get_attr(pulse, 'IC_P_Q1', 't_start')
        t_stop = db.get_attr(pulse, 'IC_P_Q1', 't_stop')
        
        if (t_start > 0) and (t_stop > 0):
            # filter the signals to the time of interest
            _P_IC, _t_P_IC = in_between(P_IC, t_P_IC, t_start, t_stop)
            _Rc,   _t_Rc   = in_between(Rc, t_Rc, t_start, t_stop)
            _V7,  _t_V7  = in_between(V7, t_V7, t_start, t_stop)
            _V9,  _t_V9  = in_between(V9, t_V9, t_start, t_stop)
            _V10,  _t_V10  = in_between(V10, t_V10, t_start, t_stop)
            _V11,  _t_V11  = in_between(V11, t_V11, t_start, t_stop)
            _V21,  _t_V21  = in_between(V21, t_V21, t_start, t_stop)
            _V7_before,  _t_V7_before  = in_between(V7, t_V7, t_start - DELTA_T_BEFORE, t_stop - DELTA_T_BEFORE)
            _V10_before,  _t_V10_before  = in_between(V10, t_V10, t_start - DELTA_T_BEFORE, t_stop - DELTA_T_BEFORE)
            _V11_before,  _t_V11_before  = in_between(V11, t_V11, t_start - DELTA_T_BEFORE, t_stop - DELTA_T_BEFORE)
            _V21_before,  _t_V21_before  = in_between(V21, t_V21, t_start - DELTA_T_BEFORE, t_stop - DELTA_T_BEFORE)
            _nl, _t_nl = in_between(nl, t_nl, t_start, t_stop)
            _ip, _t_ip = in_between(ip, t_ip, t_start, t_stop)
            
            _P_LH, _t_P_LH = in_between(P_LH, t_P_LH, t_start, t_stop)
            _Rext_upper, _t_Rext_upper = in_between(Rext_upper, t_Rext_upper, t_start, t_stop)
            _Rext_median, _t_Rext_median = in_between(Rext_median, t_Rext_median, t_start, t_stop)
            _Rext_lower, _t_Rext_lower = in_between(Rext_lower, t_Rext_lower, t_start, t_stop)
            _Zgeo, _t_Zgeo = in_between(Zgeo, t_Zgeo, t_start, t_stop)
            
            _C_LU, _t_C_LU = in_between(C_LU, t_C_LU, t_start, t_stop)
            _C_LL, _t_C_LL = in_between(C_LL, t_C_LL, t_start, t_stop)
            _C_RU, _t_C_RU = in_between(C_RU, t_C_RU, t_start, t_stop)
            _C_RL, _t_C_RL = in_between(C_RL, t_C_RL, t_start, t_stop)
            # did we used automatic matching? Check if capacitors are all almost constant. If yes, means no auto matching used for this shot
            auto_matching = not (np.allclose(_C_LU, _C_LU[0]) and np.allclose(_C_LL, _C_LL[0]) and np.allclose(_C_RU, _C_RU[0]) and np.allclose(_C_RL, _C_RL[0]))

            _Lang1, _t_Lang1 = in_between(Lang1, t_Lang1, t_start, t_stop)
            _Lang2, _t_Lang2 = in_between(Lang2, t_Lang2, t_start, t_stop)
            _Lang3, _t_Lang3 = in_between(Lang3, t_Lang3, t_start, t_stop)
            _Lang4, _t_Lang4 = in_between(Lang4, t_Lang4, t_start, t_stop)
            _Lang5, _t_Lang5 = in_between(Lang5, t_Lang5, t_start, t_stop)
            _Lang6, _t_Lang6 = in_between(Lang6, t_Lang6, t_start, t_stop)
            _Lang7, _t_Lang7 = in_between(Lang7, t_Lang7, t_start, t_stop)
            _Lang8, _t_Lang8 = in_between(Lang8, t_Lang8, t_start, t_stop)
            
            # split signals in little pieces
            nb_pieces = np.round((_t_P_IC[-1] - _t_P_IC[0])/dt)
            _P_IC_mean_min_maxs, _t_P_ICs = split_in_pieces(_P_IC, _t_P_IC, nb_pieces)
            _Rc_mean_min_maxs, _ = split_in_pieces(_Rc, _t_Rc, nb_pieces)
            _V7_mean_min_maxs, _ = split_in_pieces(_V7, _t_V7, nb_pieces)
            _V9_mean_min_maxs, _ = split_in_pieces(_V9, _t_V9, nb_pieces)
            _V10_mean_min_maxs, _ = split_in_pieces(_V10, _t_V10, nb_pieces)
            _V11_mean_min_maxs, _ = split_in_pieces(_V11, _t_V11, nb_pieces)
            _V21_mean_min_maxs, _ = split_in_pieces(_V21, _t_V21, nb_pieces)
            _V7_before_mean_min_maxs, _ = split_in_pieces(_V7_before, _t_V7_before, nb_pieces)
            _V10_before_mean_min_maxs, _ = split_in_pieces(_V10_before, _t_V10_before, nb_pieces)
            _V11_before_mean_min_maxs, _ = split_in_pieces(_V11_before, _t_V11_before, nb_pieces)
            _V21_before_mean_min_maxs, _ = split_in_pieces(_V21_before, _t_V21_before, nb_pieces)
            _nl_mean_min_max, _ = split_in_pieces(_nl, _t_nl, nb_pieces)
            _ip_mean_min_max, _ = split_in_pieces(_ip, _t_ip, nb_pieces)
            
            _P_LH_mean_min_maxs, _ = split_in_pieces(_P_LH, _t_P_LH, nb_pieces)
            _Rext_upper_mean_min_maxs, _ = split_in_pieces(_Rext_upper, _t_Rext_upper, nb_pieces)
            _Rext_median_mean_min_maxs, _ = split_in_pieces(_Rext_median, _t_Rext_median, nb_pieces)
            _Rext_lower_mean_min_maxs, _ = split_in_pieces(_Rext_lower, _t_Rext_lower, nb_pieces)
            _Zgeo_mean_min_maxs, _ = split_in_pieces(_Zgeo, _t_Zgeo, nb_pieces)            

            _Lang1_avg, _ =  split_in_pieces(_Lang1, _t_Lang1, nb_pieces)
            _Lang2_avg, _ =  split_in_pieces(_Lang2, _t_Lang2, nb_pieces)
            _Lang3_avg, _ =  split_in_pieces(_Lang3, _t_Lang3, nb_pieces)
            _Lang4_avg, _ =  split_in_pieces(_Lang4, _t_Lang4, nb_pieces)
            _Lang5_avg, _ =  split_in_pieces(_Lang5, _t_Lang5, nb_pieces)
            _Lang6_avg, _ =  split_in_pieces(_Lang6, _t_Lang6, nb_pieces)
            _Lang7_avg, _ =  split_in_pieces(_Lang7, _t_Lang7, nb_pieces)
            _Lang8_avg, _ =  split_in_pieces(_Lang8, _t_Lang8, nb_pieces)
            
            # add signals to database
            if not np.any(np.isnan(_P_IC_mean_min_maxs)):
                rows = {'pulse': pulse,
                        't': _t_P_ICs, 
                        'P_IC_avg': _P_IC_mean_min_maxs[:,0],                    
                        'Rc_avg': _Rc_mean_min_maxs[:,0],
                        'V7_avg': _V7_mean_min_maxs[:,0],
                        'V9_avg': _V9_mean_min_maxs[:,0],
                        'V10_avg': _V10_mean_min_maxs[:,0],
                        'V11_avg': _V11_mean_min_maxs[:,0],
                        'V21_avg': _V21_mean_min_maxs[:,0],
                        'V7_before_avg': _V7_before_mean_min_maxs[:,0],
                        'V10_before_avg': _V10_before_mean_min_maxs[:,0],
                        'V11_before_avg': _V11_before_mean_min_maxs[:,0],
                        'V21_before_avg': _V21_before_mean_min_maxs[:,0],
                        'P_LH_avg': _P_LH_mean_min_maxs[:,0],
                        'Rext_upper': _Rext_upper_mean_min_maxs[:,0],
                        'Rext_median': _Rext_median_mean_min_maxs[:,0],
                        'Rext_lower': _Rext_lower_mean_min_maxs[:,0],
                        'Zgeo': _Zgeo_mean_min_maxs[:,0],
                        'R_Q1': IC_Positions[0],
                        'R_Q2': IC_Positions[1],
                        'R_Q4': IC_Positions[2],
                        'freq_Q1': IC_Frequencies[0],
                        'freq_Q2': IC_Frequencies[1],
                        'freq_Q4': IC_Frequencies[2],
                        'gap_median': IC_Positions[0]*1e3 - _Rext_median_mean_min_maxs[:,0],
                        'nl_avg': _nl_mean_min_max[:,0],
                        'Ip_avg': _ip_mean_min_max[:,0],
                        'auto_matching': int(auto_matching),
                        'Lang1_avg': _Lang1_avg[:,0], 'Lang2_avg': _Lang2_avg[:,0], 'Lang3_avg': _Lang3_avg[:,0], 'Lang4_avg': _Lang4_avg[:,0],
                        'Lang5_avg': _Lang5_avg[:,0], 'Lang6_avg': _Lang6_avg[:,0], 'Lang7_avg': _Lang7_avg[:,0], 'Lang8_avg': _Lang8_avg[:,0],
                       }
                df = pd.DataFrame(rows)
                
                data_Q1 = data_Q1.append(df)
            

    except KeyError as e:
        pass


 53%|█████████████████████▌                   | 69/131 [00:12<00:09,  6.53it/s]C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|████████████████████████████████████████| 131/131 [00:20<00:00, 10.75it/s]


In [7]:
DELTA_T_BEFORE = 1  # s

dt = 0.05  # s

data_Q2 = pd.DataFrame()

# generate averaged values for Q2
for pulse in tqdm(db.pulse_list):
    text = pulse
    try:
        P_IC, t_P_IC = db.get_signal(pulse, 'IC_P_Q2')
        Rc,   t_Rc   = db.get_signal(pulse, 'IC_Rc_Q2_avg')
        V7,  t_V7  = db.get_signal(pulse, 'Valve7')
        V9,  t_V9  = db.get_signal(pulse, 'Valve9')
        V10,  t_V10  = db.get_signal(pulse, 'Valve10')
        V11,  t_V11  = db.get_signal(pulse, 'Valve11')
        V21,  t_V21  = db.get_signal(pulse, 'Valve21')
        nl, t_nl = db.get_signal(pulse, 'nl')
        ip, t_ip = db.get_signal(pulse, 'Ip')
        
        Rext_upper, t_Rext_upper = db.get_signal(pulse, 'Rext_upper')
        Rext_median, t_Rext_median = db.get_signal(pulse, 'Rext_median')
        Rext_lower, t_Rext_lower = db.get_signal(pulse, 'Rext_lower')
        Zgeo, t_Zgeo = db.get_signal(pulse, 'Zgeo')
        IC_Positions, _ = db.get_signal(pulse, 'IC_Positions')
        IC_Frequencies, _ = np.round(db.get_signal(pulse, 'IC_Frequencies'), decimals=1)
        
        try: # il n'y a pas forcément de P_LH
            P_LH, t_P_LH = db.get_signal(pulse, 'LH_P_tot')
            if np.any(np.isnan(P_LH)):
                P_LH = np.zeros_like(P_IC)
                t_P_LH = t_P_IC
        except:
            P_LH = np.zeros_like(P_IC)
            t_P_LH = t_P_IC
        
        C_LU, t_C_LU = db.get_signal(pulse, 'IC_Capa_Q2_left_upper')
        C_LL, t_C_LL = db.get_signal(pulse, 'IC_Capa_Q2_left_lower')
        C_RU, t_C_RU = db.get_signal(pulse, 'IC_Capa_Q2_right_upper')
        C_RL, t_C_RL = db.get_signal(pulse, 'IC_Capa_Q2_right_lower')

        try:
            Lang1, t_Lang1 = db.get_signal(pulse, 'Langmuir_LHCD1')
            Lang2, t_Lang2 = db.get_signal(pulse, 'Langmuir_LHCD2')
            Lang3, t_Lang3 = db.get_signal(pulse, 'Langmuir_LHCD3')
            Lang4, t_Lang4 = db.get_signal(pulse, 'Langmuir_LHCD4')
            Lang5, t_Lang5 = db.get_signal(pulse, 'Langmuir_LHCD5')
            Lang6, t_Lang6 = db.get_signal(pulse, 'Langmuir_LHCD6')
            Lang7, t_Lang7 = db.get_signal(pulse, 'Langmuir_LHCD7')
            Lang8, t_Lang8 = db.get_signal(pulse, 'Langmuir_LHCD8')
        except:
            Lang1, t_Lang1 = np.zeros_like(P_IC), t_P_IC
            Lang2, t_Lang2 = np.zeros_like(P_IC), t_P_IC
            Lang3, t_Lang3 = np.zeros_like(P_IC), t_P_IC
            Lang4, t_Lang4 = np.zeros_like(P_IC), t_P_IC
            Lang5, t_Lang5 = np.zeros_like(P_IC), t_P_IC
            Lang6, t_Lang6 = np.zeros_like(P_IC), t_P_IC
            Lang7, t_Lang7 = np.zeros_like(P_IC), t_P_IC
            Lang8, t_Lang8 = np.zeros_like(P_IC), t_P_IC
            
        t_start = db.get_attr(pulse, 'IC_P_Q2', 't_start')
        t_stop = db.get_attr(pulse, 'IC_P_Q2', 't_stop')
        
        if (t_start > 0) and (t_stop > 0):
            # filter the signals to the time of interest
            _P_IC, _t_P_IC = in_between(P_IC, t_P_IC, t_start, t_stop)
            _P_LH, _t_P_LH = in_between(P_LH, t_P_LH, t_start, t_stop)
            _Rc,   _t_Rc   = in_between(Rc, t_Rc, t_start, t_stop)
            _V7,  _t_V7  = in_between(V7, t_V7, t_start, t_stop)
            _V9,  _t_V9  = in_between(V9, t_V9, t_start, t_stop)
            _V10,  _t_V10  = in_between(V10, t_V10, t_start, t_stop)
            _V11,  _t_V11  = in_between(V11, t_V11, t_start, t_stop)
            _V21,  _t_V21  = in_between(V21, t_V21, t_start, t_stop)
            _V7_before,  _t_V7_before  = in_between(V7, t_V7, t_start - DELTA_T_BEFORE, t_stop - DELTA_T_BEFORE)
            _V10_before,  _t_V10_before  = in_between(V10, t_V10, t_start - DELTA_T_BEFORE, t_stop - DELTA_T_BEFORE)
            _V11_before,  _t_V11_before  = in_between(V11, t_V11, t_start - DELTA_T_BEFORE, t_stop - DELTA_T_BEFORE)
            _V21_before,  _t_V21_before  = in_between(V21, t_V21, t_start - DELTA_T_BEFORE, t_stop - DELTA_T_BEFORE)
            _nl, _t_nl = in_between(nl, t_nl, t_start, t_stop)
            _ip, _t_ip = in_between(ip, t_ip, t_start, t_stop)
            
            _Rext_upper, _t_Rext_upper = in_between(Rext_upper, t_Rext_upper, t_start, t_stop)
            _Rext_median, _t_Rext_median = in_between(Rext_median, t_Rext_median, t_start, t_stop)
            _Rext_lower, _t_Rext_lower = in_between(Rext_lower, t_Rext_lower, t_start, t_stop)
            _Zgeo, _t_Zgeo = in_between(Zgeo, t_Zgeo, t_start, t_stop)

            _C_LU, _t_C_LU = in_between(C_LU, t_C_LU, t_start, t_stop)
            _C_LL, _t_C_LL = in_between(C_LL, t_C_LL, t_start, t_stop)
            _C_RU, _t_C_RU = in_between(C_RU, t_C_RU, t_start, t_stop)
            _C_RL, _t_C_RL = in_between(C_RL, t_C_RL, t_start, t_stop)
            # did we used automatic matching? Check if capacitors are all almost constant. If yes, means no auto matching used for this shot
            auto_matching = not (np.allclose(_C_LU, _C_LU[0]) and np.allclose(_C_LL, _C_LL[0]) and np.allclose(_C_RU, _C_RU[0]) and np.allclose(_C_RL, _C_RL[0]))
            
            _Lang1, _t_Lang1 = in_between(Lang1, t_Lang1, t_start, t_stop)
            _Lang2, _t_Lang2 = in_between(Lang2, t_Lang2, t_start, t_stop)
            _Lang3, _t_Lang3 = in_between(Lang3, t_Lang3, t_start, t_stop)
            _Lang4, _t_Lang4 = in_between(Lang4, t_Lang4, t_start, t_stop)
            _Lang5, _t_Lang5 = in_between(Lang5, t_Lang5, t_start, t_stop)
            _Lang6, _t_Lang6 = in_between(Lang6, t_Lang6, t_start, t_stop)
            _Lang7, _t_Lang7 = in_between(Lang7, t_Lang7, t_start, t_stop)
            _Lang8, _t_Lang8 = in_between(Lang8, t_Lang8, t_start, t_stop)
            
            # split signals in little pieces
            nb_pieces = np.round((_t_P_IC[-1] - _t_P_IC[0])/dt)
            _P_IC_mean_min_maxs, _t_P_ICs = split_in_pieces(_P_IC, _t_P_IC, nb_pieces)
            _P_LH_mean_min_maxs, _ = split_in_pieces(_P_LH, _t_P_LH, nb_pieces)
            _Rc_mean_min_maxs, _ = split_in_pieces(_Rc, _t_Rc, nb_pieces)
            _V7_mean_min_maxs, _ = split_in_pieces(_V7, _t_V7, nb_pieces)
            _V9_mean_min_maxs, _ = split_in_pieces(_V9, _t_V9, nb_pieces)
            _V10_mean_min_maxs, _ = split_in_pieces(_V10, _t_V10, nb_pieces)
            _V11_mean_min_maxs, _ = split_in_pieces(_V11, _t_V11, nb_pieces)
            _V21_mean_min_maxs, _ = split_in_pieces(_V21, _t_V21, nb_pieces)
            _V7_before_mean_min_maxs, _ = split_in_pieces(_V7_before, _t_V7_before, nb_pieces)
            _V10_before_mean_min_maxs, _ = split_in_pieces(_V10_before, _t_V10_before, nb_pieces)
            _V11_before_mean_min_maxs, _ = split_in_pieces(_V11_before, _t_V11_before, nb_pieces)
            _V21_before_mean_min_maxs, _ = split_in_pieces(_V21_before, _t_V21_before, nb_pieces)
            _nl_mean_min_max, _ = split_in_pieces(_nl, _t_nl, nb_pieces)
            _ip_mean_min_max, _ = split_in_pieces(_ip, _t_ip, nb_pieces)
            
            _Rext_upper_mean_min_maxs, _ = split_in_pieces(_Rext_upper, _t_Rext_upper, nb_pieces)
            _Rext_median_mean_min_maxs, _ = split_in_pieces(_Rext_median, _t_Rext_median, nb_pieces)
            _Rext_lower_mean_min_maxs, _ = split_in_pieces(_Rext_lower, _t_Rext_lower, nb_pieces)
            _Zgeo_mean_min_maxs, _ = split_in_pieces(_Zgeo, _t_Zgeo, nb_pieces)
            
            _Lang1_avg, _ =  split_in_pieces(_Lang1, _t_Lang1, nb_pieces)
            _Lang2_avg, _ =  split_in_pieces(_Lang2, _t_Lang2, nb_pieces)
            _Lang3_avg, _ =  split_in_pieces(_Lang3, _t_Lang3, nb_pieces)
            _Lang4_avg, _ =  split_in_pieces(_Lang4, _t_Lang4, nb_pieces)
            _Lang5_avg, _ =  split_in_pieces(_Lang5, _t_Lang5, nb_pieces)
            _Lang6_avg, _ =  split_in_pieces(_Lang6, _t_Lang6, nb_pieces)
            _Lang7_avg, _ =  split_in_pieces(_Lang7, _t_Lang7, nb_pieces)
            _Lang8_avg, _ =  split_in_pieces(_Lang8, _t_Lang8, nb_pieces)
            
            # add signals to database
            if not np.any(np.isnan(_P_IC_mean_min_maxs)):
                rows = {'pulse': pulse,
                        't': _t_P_ICs, 
                        'P_IC_avg': _P_IC_mean_min_maxs[:,0],                    
                        'Rc_avg': _Rc_mean_min_maxs[:,0],
                        'V7_avg': _V7_mean_min_maxs[:,0],
                        'V9_avg': _V9_mean_min_maxs[:,0],
                        'V10_avg': _V10_mean_min_maxs[:,0],
                        'V11_avg': _V11_mean_min_maxs[:,0],
                        'V21_avg': _V21_mean_min_maxs[:,0],
                        'V7_before_avg': _V7_before_mean_min_maxs[:,0],
                        'V10_before_avg': _V10_before_mean_min_maxs[:,0],
                        'V11_before_avg': _V11_before_mean_min_maxs[:,0],
                        'V21_before_avg': _V21_before_mean_min_maxs[:,0],
                        'P_LH_avg': _P_LH_mean_min_maxs[:,0],
                        'Rext_upper': _Rext_upper_mean_min_maxs[:,0],
                        'Rext_median': _Rext_median_mean_min_maxs[:,0],
                        'Rext_lower': _Rext_lower_mean_min_maxs[:,0],
                        'Zgeo': _Zgeo_mean_min_maxs[:,0],
                        'R_Q1': IC_Positions[0],
                        'R_Q2': IC_Positions[1],
                        'R_Q4': IC_Positions[2],
                        'freq_Q1': IC_Frequencies[0],
                        'freq_Q2': IC_Frequencies[1],
                        'freq_Q4': IC_Frequencies[2],
                        'gap_median': IC_Positions[1]*1e3 - _Rext_median_mean_min_maxs[:,0],
                        'nl_avg': _nl_mean_min_max[:,0],
                        'Ip_avg': _ip_mean_min_max[:,0],
                        'auto_matching': int(auto_matching),
                        'Lang1_avg': _Lang1_avg[:,0], 'Lang2_avg': _Lang2_avg[:,0], 'Lang3_avg': _Lang3_avg[:,0], 'Lang4_avg': _Lang4_avg[:,0],
                        'Lang5_avg': _Lang5_avg[:,0], 'Lang6_avg': _Lang6_avg[:,0], 'Lang7_avg': _Lang7_avg[:,0], 'Lang8_avg': _Lang8_avg[:,0],
                       }
                df = pd.DataFrame(rows)
                
                data_Q2 = data_Q2.append(df)
            

    except KeyError as e:
        pass


100%|████████████████████████████████████████| 131/131 [00:20<00:00,  6.95it/s]


In [8]:
data_Q1.to_csv('WEST_C3b_database_resumed_parameters_Q1.csv')
data_Q2.to_csv('WEST_C3b_database_resumed_parameters_Q2.csv')